In [1]:
import pandas as pd
import numpy as np
import sys, re, requests, collections, glob, time 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
import json
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
%matplotlib inline

In [55]:
sub = pd.read_csv('SampleSubmissionStage1.csv')
sub.head()

data_dict = {}
for i in glob.glob('data/*'):
    name = i.split('/')[-1].split('.')[0]
    print(i)
    if name != 'TeamSpellings':
        data_dict[name] = pd.read_csv(i)
    else:
        data_dict[name] = pd.read_csv(i, encoding='latin-1')        

print('team_ranks/MasseyOrdinals.csv')
data_dict["OrdinalRankings"] = pd.read_csv('team_ranks/MasseyOrdinals.csv') #Use to generate quality wins statistics

data/GameCities.csv
data/NCAATourneySlots.csv
data/SecondaryTourneyCompactResults.csv
data/Cities.csv
data/SecondaryTourneyTeams.csv
data/RegularSeasonDetailedResults.csv
data/RegularSeasonCompactResults.csv
data/Teams.csv
data/Conferences.csv
data/NCAATourneyCompactResults.csv
data/NCAATourneySeeds.csv
data/TeamSpellings.csv
data/NCAATourneySeedRoundSlots.csv
data/TeamCoaches.csv
data/Seasons.csv
data/ConferenceTourneyGames.csv
data/NCAATourneyDetailedResults.csv
data/TeamConferences.csv


In [4]:
data_dict['NCAATourneySeeds']['Seed'] = data_dict['NCAATourneySeeds']['Seed'].apply(lambda x: int(x[1:3]))
data_dict['NCAATourneySeeds'] = data_dict['NCAATourneySeeds'][['Season', 'TeamID', 'Seed']]
data_dict['NCAATourneySeeds'].head()

,Season,TeamID,Seed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [5]:
data_dict['NCAATourneyCompactResults'] = data_dict['NCAATourneyCompactResults'][['Season','WTeamID', 'LTeamID']]
data_dict['NCAATourneyCompactResults'].head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [6]:
df = pd.merge(data_dict['NCAATourneyCompactResults'], data_dict['NCAATourneySeeds'], how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df = pd.merge(df, data_dict['NCAATourneySeeds'], how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df = df.drop(['TeamID_x', 'TeamID_y'], axis=1)
df['seed_diff'] = df['Seed_x'] - df['Seed_y']
df.head()

,Season,WTeamID,LTeamID,Seed_x,Seed_y,seed_diff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11


In [ ]:
#Prediction of Models is based upon Log Loss 

#This will be sklearn.metric.log_loss(y_true, y_pred)

In [40]:
#new_df is training data, sub is submission data

X = new_df.drop(['target'], axis=1)
scaler = sklearn.preprocessing.StandardScaler()
y = new_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [53]:
%%time
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
pred = reg.predict(X_train).clip(0,1)
print(log_loss(y_train, pred))


0.10352726224307937
CPU times: user 16 ms, sys: 3 ms, total: 19 ms
Wall time: 14.3 ms
